### 0. install requirements

In [ ]:
%pip install -r requirements.txt

### 1. Testing Model *GPT2*

Use model with ***pipeline*** as a high-level helper.  
you can see the model detaile [here](https://huggingface.co/openai-community/gpt2).

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

pipe("The future of AI is ", max_length=50, num_return_sequences=1, max_new_tokens=10)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "The future of AI is \xa0beyond the human brain. It's coming"}]

the answers are false. for example:
'US is a country in ʻal-Arabia, the region with' or 'The future of AI is \xa0beyond the human brain. It's coming'

### 2. Fine-Tuning

Here we do ***Fine-Tuning*** with datasets ***wikitext***.

#### 2-1. load datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
dataset = dataset.select(range(200)) # for fast running

print(dataset)

Column(['', ' = Valkyria Chronicles III = \n', '', ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game mor